In [1]:
# query is a track 
# query is a playlist 
# suggestion is always a track


In [1]:
import json
import numpy as np
import sklearn
import sklearn.metrics
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import os
import snap
data_dir = "/Users/pani/mpd/data/"

In [2]:
data_files = list(sorted(os.listdir(data_dir)))

In [3]:
data_files = data_files[:10]

In [4]:
all_playlists = []
for file_number, data_file in enumerate(data_files):
#     print(file_number, data_file)
    with open(data_dir + data_file) as file:
        data = json.load(file)
        playlists = data['playlists']
        all_playlists += playlists

In [5]:
playlist_list = all_playlists
len(playlist_list)

10000

In [6]:
import sys
sys.getsizeof(all_playlists)

90120

In [7]:
def get_tracks(playlist):
    track_list = set([])
    for i in range(playlist['num_tracks']):
        track_list.add(playlist['tracks'][i]['track_uri'])
    return track_list

In [8]:
# tracks = get_tracks(data['playlists'][0])
# tracks

In [9]:
# A graph is made in which an edge between playlist-id and track-id is missing
# the aim is to predict the missing track in the playlist

# g, dropped_track_id = make_graph_drop_edge(playlist_list, playlist_id, dropped_track_index)/

# This function creats the track ids

def make_graph_drop_edge(playlist_list, playlist_id, dropped_track_index):
# What if a track was common between two playlist? If there is no dictionary, track+id +=1 produces
# 2 node ids for the same track.
    g = snap.TUNGraph.New()
    next_track_id = len(playlist_list)
    tracks_dict = {}
    dropped_track_id = None
    for increasing_playlist_id in range(len(playlist_list)):
        g.AddNode(increasing_playlist_id)
        track_list = get_tracks(playlist_list[increasing_playlist_id])
        track_list = sorted(track_list)
        
        for track_uri in track_list:
            track_index = track_list.index(track_uri)
#             to have unique track id
            if track_uri in tracks_dict:
                increasing_track_id = tracks_dict[track_uri]
            else:
                tracks_dict[track_uri] = next_track_id
                next_track_id += 1
                increasing_track_id = tracks_dict[track_uri]
                g.AddNode(increasing_track_id)
            if (playlist_id == increasing_playlist_id and track_index == dropped_track_index):
                dropped_track_id = increasing_track_id
                continue
            g.AddEdge(increasing_track_id, increasing_playlist_id)
    return g, dropped_track_id

In [10]:
# list(g.GetNI(0).GetOutEdges())

In [11]:
# Basic random walk from playlist_id to track_id

from collections import defaultdict
import random

def basic_random_walk_playlist(query,g):
    N = 10000
    totSteps = 0
    num_visits = defaultdict(int)

    while totSteps < N : 
        currPlaylist = query
        # Number of iterations
        currSteps = 5
        for i in range(currSteps):
            # takes 1 step (from a playlist to track)
            edges = list(g.GetNI(currPlaylist).GetOutEdges())
            currTrack = random.choice(edges)
            # takes 1 step (from a track to playlist)
            edges = list(g.GetNI(currTrack).GetOutEdges())
            currPlaylist = random.choice(edges)
            num_visits[currTrack] += 1
        totSteps += currSteps
    return num_visits

In [12]:
# edges = list(g.GetNI(0).GetOutEdges())
# print edges

In [13]:
# g = make_graph_drop_edge(playlist_list, 20, 1020)
# edges = list(g.GetNI(0).GetOutEdges())
# print edges

In [14]:
# all_true_indices : list of indices that I dropped
# all_indices: list of list of best suggested track ids


# evaluation metric
def cal_ave_precision(k, all_true_indices, all_indices):
    sum_precision = 0.0
    for i in range(len(all_true_indices)):
        precision = float(all_true_indices[i] in all_indices[i][:k])
        sum_precision += precision
    average_precision = sum_precision / len(all_true_indices)
    return average_precision

In [15]:
# dropped_track_id = 2000

# g = make_graph_drop_edge(playlist_list, 500, dropped_track_id)
# print g

In [16]:
# num_visits = basic_random_walk_playlist(query,g)
# num_visits

In [17]:
# all_true_indices : list of indices that I dropped
# all_indices: list of list of best suggested track ids
# all_true_indices = [dropped_track_id]
# all_indices = [sorted(num_visits, key=num_visits.get, reverse=True)[:5]]
# avg_precision = cal_ave_precision(2, all_true_indices, all_indices)
# print avg_precision

In [18]:
# all_true_indices : list of ids that I dropped
# all_indices: list of list of best suggested track ids

# dropped_track_id = 2000

# g = make_graph_drop_edge(playlist_list, 500, dropped_track_id)
# print g

# num_visits = basic_random_walk_playlist(query,g)
# num_visits
from tqdm import tqdm_notebook as tqdm

def cal_most_visited(playlist_list):
#     correct track ids
    all_true_ids = []
    all_suggested_ids = []
    for playlist_id in tqdm(range(len(playlist_list))):
        dropped_track_index = random.choice(range(len(playlist_list[playlist_id])))
#         for dropped_track_index in range(len(playlist_list[playlist_id])):
        g, dropped_track_id = make_graph_drop_edge(playlist_list, playlist_id, dropped_track_index)
#             print('1', playlist_id, dropped_track_id)
#             tracks in the playlist graphs where one is dropped
        if len(list(g.GetNI(playlist_id).GetOutEdges())) < 5:
            continue
        num_visits = basic_random_walk_playlist(playlist_id, g)
#             print num_visits
        indices = sorted(num_visits, key=num_visits.get, reverse=True)
        all_true_ids.append(dropped_track_id)
        all_suggested_ids.append(indices)
    return all_true_ids, all_suggested_ids

In [19]:
import multiprocessing
import functools
from tqdm import tqdm_notebook as tqdm

def cal_most_visited_playlist(playlist_list, playlist_id):
    dropped_track_index = random.choice(range(len(playlist_list[playlist_id])))
#         for dropped_track_index in range(len(playlist_list[playlist_id])):
    g, dropped_track_id = make_graph_drop_edge(playlist_list, playlist_id, dropped_track_index)
#             print('1', playlist_id, dropped_track_id)
#             tracks in the playlist graphs where one is dropped
    if len(list(g.GetNI(playlist_id).GetOutEdges())) < 5:
        return None, None
    num_visits = basic_random_walk_playlist(playlist_id, g)
    suggested_ids = sorted(num_visits, key=num_visits.get, reverse=True)
    return dropped_track_id, suggested_ids

def process_playlists(fn, playlist_list):
    pool = multiprocessing.Pool(3)
    num_playlist = len(playlist_list)
    fn_with_playlist_list = functools.partial(fn, playlist_list)
    all_true_ids = []
    all_suggested_ids = []
#     map(fn, [1,2,3,4]) -> [fn(1), fn(2), fn(3), fn(4)]
    result = pool.imap_unordered(fn_with_playlist_list, range(num_playlist),
                                 chunksize=10)
    for true_id, suggested_ids in tqdm(result, total=num_playlist):
        if true_id is None:
            continue
        all_true_ids.append(true_id)
        all_suggested_ids.append(suggested_ids)
    return all_true_ids, all_suggested_ids

In [20]:
# all_true_ids, all_suggested_ids = cal_most_visited(playlist_list)
all_true_ids, all_suggested_ids = process_playlists(cal_most_visited_playlist, playlist_list)
avg_precision = cal_ave_precision(500, all_true_ids, all_suggested_ids)
print avg_precision


0.335774590581


In [242]:
all_true_indices[0] in all_indices[0]

True

In [246]:
len(all_indices[0])

6808

In [243]:
all_true_indices[0]

1000

In [ ]:
# g = make_graph_drop_edge(playlist_list, 0, 1000)
# num_visits = basic_random_walk_playlist(0,g)
# print num_visits

In [224]:
1000 in num_visits.keys()

True

In [225]:
num_visits[1000]

5

In [226]:
max(num_visits.values())

143

In [227]:
list(sorted(num_visits, key=num_visits.get, reverse=True)).index(1000)

500

In [228]:
list(g.GetNI(1000).GetOutEdges())

[359, 812, 908]

In [229]:
snap.GetShortPath(g, 0, 1000)

3

In [230]:
g.IsEdge(0, 1000)

False

In [232]:
cal_ave_precision(600, [1000], [num_visits.keys()])

1.0

In [150]:
# evaluation metric
def cal_ave_precision(k, all_true_indices, all_indices):
    sum_precision = 0.0
    for i in range(len(all_true_indices)):
        precision = float(all_true_indices[i] in all_indices[i][:k])
        sum_precision += precision
    average_precision = sum_precision / len(all_true_indices)
    return average_precision

In [153]:
float(False)

0.0

In [72]:
import pdb; pdb.pm()

> <ipython-input-70-046a38a27c48>(21)cal_most_visited()
-> if len(g.GetNI(playlist_id).GetOutEdges()) < 5:
(Pdb) q


In [122]:
g = make_graph_drop_edge(playlist_list, 0, 1000)
g.GetEdges()

66720

In [123]:
num_visits = basic_random_walk_playlist(0,g)
num_visits

defaultdict(int,
            {1001: 1,
             1002: 1,
             1013: 1,
             1024: 1,
             1039: 1,
             4737: 1,
             6607: 1,
             6677: 1,
             14055: 1})